In [ ]:
%%capture

methods = ['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'dpli', 'wpli', 'wpli2_debiased'] #removed cohy because it's not there in results folder

data_types = ['REM', 'N1', 'N2', 'N3', 'Wake']
expt_num = 1
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/Connectivity_Results/Experiment_set_2/'

#Try loading a results file
method = methods[0]
data_type = data_types[1]

# Create summary results dictionary that will be filled and populated
summary_results_dict = {}
summary_results_dict['metric'] = []
for data_type in data_types:
    summary_results_dict[data_type + '_model'] = []
    summary_results_dict[data_type + '_acc'] = []
    summary_results_dict[data_type + '_acc' + '_std'] = []

#Dictionary to store raw results for calculation of distribution(s) and statistical tests 
raw_results = {}
for method in methods:
    raw_results[method] = {}


#error dict can catch those metrics / data types for which we don't have results files
error_dict = {}
for method in methods:
    error_dict[method] = []
    
    summary_results_dict['metric'].append(method)
    
    for data_type in data_types:
        try:
            load_path = save_folder + data_type + method + '_expt_' + str(expt_num) + '_' + 'raw_clfs_results.h5'
            
            results_df = pd.read_hdf(load_path)
            results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]
            
            raw_results[method][data_type] = results_df

        
            
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
            # This will get you best performing Ada Model and Value for each method for each data type
            best_acc = res_df.mean(axis = 0)[1:].max()
            best_model = res_df.mean(axis = 0)[1:].idxmax()

            #Use the summary res_df to identify best model, keep raw results of best model to calculate the true std
            results_df_trimmed = results_df[ list(results_df.columns)[:2] + [col for col in results_df.columns if best_model[:-4] in col] ]
            per_fold_metrics = calculate_results.calc_per_fold_metrics(results_df)
            overalled_weighted_df = calculate_results.overall_weighted_metrics(per_fold_metrics)
            
            std_val = overalled_weighted_df[best_model + '_std'].values[0]

            summary_results_dict[data_type + '_model'].append(best_model[:-10])
            summary_results_dict[data_type + '_acc'].append(best_acc)
            summary_results_dict[data_type + '_acc' + '_std'].append(std_val)
            
        except:
            error_dict[method].append(data_type)